In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
mistral_api_key = os.environ["MISTRAL_API_KEY"]

In [2]:
from langchain_mistralai import ChatMistralAI

chatModel = ChatMistralAI(
    mistral_api_key=mistral_api_key,
    model="mistral-large-latest",
    temperature=1,
    max_retries=2,
    # other params...
)

## Load data from Wiki

In [3]:
from langchain_community.document_loaders import WikipediaLoader

loader = WikipediaLoader(query="Amitabh Bacchan", load_max_docs=2)

loaded_data = loader.load()

In [4]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

## Split the data

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=500)

chunks_of_text = text_splitter.split_documents(loaded_data)


Created a chunk of size 2079, which is longer than the specified 1000


### Use FAISS Vector Database

In [6]:
vector_db = FAISS.from_documents(chunks_of_text, 
                                 embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"),
                                )

/Users/pramodmodi/Library/Caches/pypoetry/virtualenvs/langchain-mistral-xuCIEomg-py3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
retriever = vector_db.as_retriever(search_kwargs={"k": 1})

### Use chain to pass context and question to llm and then parse the result

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
#chat_template = ChatPromptTemplate.from_messages(template)


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chatModel
    | StrOutputParser()
)

response = chain.invoke("When and where Mr Bacchan born?")

In [ ]:
response

In [62]:
query= "What are famous movies done by Amitabh Bachhan?"
for chunk in chain.stream(query):
    print(chunk, end="", flush=True)

Based on the context, some of the famous movies done by Amitabh Bachchan include:

From the 1970s to 1980s:
- Anand
- Zanjeer
- Roti Kapada Aur Makaan
- Deewaar
- Sholay
- Kabhi Kabhie
- Hera Pheri
- Amar Akbar Anthony
- Parvarish
- Khoon Pasina
- Kasme Vaade
- Don
- Trishul
- Muqaddar Ka Sikandar
- Suhaag
- Dostana
- Naseeb
- Laawaris
- Namak Halaal
- Coolie
- Sharaabi
- Mard
- Namak Haraam
- Abhimaan
- Majboor
- Mili
- Chupke Chupke
- Kaala Patthar
- Shaan
- Silsila
- Yaarana
- Kaalia
- Shakti
- Aakhree Raasta
- Agneepath
- Hum

From the 2000s onwards:
- Mohabbatein
- Kabhi Khushi Kabhie Gham
- Aankhen
- Baghban
- Khakee
- Black
- Bunty Aur Babli
- Sarkar
- Kabhi Alvida Naa Kehna
- Bhoothnath
- Cheeni Kum
- Paa
- Piku
- Pink
- Badla
- Brahmāstra: Part One – Shiva
- Kalki 2898 AD

He also appeared in the Hollywood film:
- The Great Gatsby (2013)